In [3]:
%load_ext autoreload
%autoreload 2

In [26]:
import fiona
import collections
import matplotlib.pyplot as plt
import pyproj
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio import features
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date, timedelta
import geopandas
import os
from shapely import geometry
import numpy as np
import utilities
import pandas as pd
import shutil
import glob
pd.set_option('display.max_columns', None)
import time
import zipfile
import geopandas as gpd

In [5]:
# 18-21 Jul 2019
start_date = date(2021, 3, 25)
end_date = date(2021, 4, 1)

In [6]:
path = "/home/primity/terras/bulk/"
dest = "/home/primity/terras/bulk_crops/"

In [7]:
os.makedirs(path, exist_ok=True)
os.makedirs(dest, exist_ok=True)
footprint = geojson_to_wkt(read_geojson('quadrado-macao.geojson'))
for current_date in utilities.daterange(start_date, end_date -  timedelta(1)):
    os.makedirs(path, exist_ok=True)
    #get products list from this day
    api = SentinelAPI('fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')
    products = api.query(footprint,
                         date=(current_date, current_date + timedelta(1)),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))
    products_df = api.to_dataframe(products)
    
    print(f"{current_date} - {len(products)} available products")
    # download them all
    break
    for p in list(products_df.index):
        utilities.download_no_fail(p, path, 'fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')
        
        with zipfile.ZipFile(os.path.join(path, products_df.loc[p]['title']+'.zip'), 'r') as zip_ref:
            zip_ref.extractall(path)
        
        os.remove(os.path.join(path, products_df.loc[p]['title']+'.zip'))
        # copy jp2 to dest
        utilities.subfolders_copy(os.path.join(path, products_df.loc[p]['title']), dest)
        break
        # save cropped jp2 with same name 
        for i in os.listdir(dest):
            with rasterio.open(os.path.join(image_dir, i)) as src:
                out_image, out_transform = rasterio.mask.mask(src, utilities.transform_shapefile(macao_shp), crop=True, nodata=10, all_touched=True)
                out_meta = src.meta.copy() 
            with rasterio.open(os.path.join(image_dir, i), 'w', **profile) as dst:
                dst.write(out_image.astype(rasterio.uint8), 1)
                
        for f in glob.glob(os.path.join(path, products_df.loc[p]['title'])):
            os.remove(f)
    

2021-03-25 - 3 available products


In [28]:
products_df['footprint'] = geopandas.GeoSeries.from_wkt(products_df['footprint'])

In [27]:
products_df =  gpd.GeoDataFrame(products_df, geometry='footprint')

TypeError: Input must be valid geometry objects: MULTIPOLYGON (((-6.555237 39.63587147039632, -6.519562 40.62420990912828, -7.8172913 40.64479964997401, -7.83432 39.65575725270223, -6.555237 39.63587147039632)))

In [34]:
for i in [0,1,2,3]:
    print((products_df.footprint[i]).centroid)

POINT (-7.181615396783749 40.14135336352725)
POINT (-8.35563334645798 40.15384073405952)
POINT (-7.181615396783749 40.14135336352725)


IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
terras/bulk/S2A_MSIL2A_20210325T112111_N0214_R037_T29TPE_20210325T140825.zip'
terras/bulk/S2A_MSIL2A_20210325T112111_N0214_R037_T29TPE_20210325T140825.zip